# Modular 구조

```python

modualer_model/ # 프로젝트 폴더
├── service/
│   ├── utils.py
│   ├── dataset.py # 데이터
│   ├── model.py # 모델
│   └── run.py # 학습 프로세스
└── data/ # 학습 데이터
    ├── train.csv
    ├── test.csv
    └── submission.csv
```

## 폴더 생성

In [ ]:
import os

for dir in ['service', 'data']:
    # 만약 해당 폴더가 없다면,
    if not os.path.exists(dir):
        # 해당 폴더를 만들어줘
        os.makedirs(dir)

## utils 생성

In [ ]:
%%writefile service/utils.py

import os
import random
import numpy as np
import torch

def reset_seeds(func, seed=42):
  random.seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)    # 파이썬 환경변수 시드 고정
  np.random.seed(seed)
  torch.manual_seed(seed) # cpu 연산 무작위 고정
  torch.cuda.manual_seed(seed) # gpu 연산 무작위 고정
  torch.backends.cudnn.deterministic = True  # cuda 라이브러리에서 Deterministic(결정론적)으로 예측하기 (예측에 대한 불확실성 제거 )

  def wrapper_func(*args, **kwargs):
    return func(*args, **kwargs)

  return wrapper_func


Overwriting service/utils.py


## dataset 생성

In [ ]:
%%writefile service/dataset.py

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from utils import reset_seeds

# 타이타닉 데이터 로드
def __load_data() -> pd.DataFrame:
    return pd.read_csv("./data/train.csv")

def __process_drop(train, test):
    drop_cols = ['name', 'ticket', 'cabin']

    train.drop(drop_cols, axis=1, inplace=True) # 모델이 학습하는데 사용하는 데이터
    test.drop(drop_cols, axis=1, inplace=True) # 모델의 학습을 평가(잘했는지?? 못했는지??)하기 위한 데이터

def __process_null(train, test):
    age_median = train['age'].median()
    fare_median = train['fare'].median()
    embarked_mode = train['embarked'].mode().values[0]

    train['age'].fillna(age_median, inplace=True)
    test['age'].fillna(age_median, inplace=True)

    train['fare'].fillna(fare_median, inplace=True)
    test['fare'].fillna(fare_median, inplace=True)

    train['embarked'].fillna(embarked_mode, inplace=True)
    test['embarked'].fillna(embarked_mode, inplace=True)


def __preprocess_data(train, test):
    print(f'before: {train.shape} / {test.shape}')
    # 필요없는 컬럼 제거
    __process_drop(train, test)
    # 결측치 처리
    __process_null(train, test)
    # 범주형 처리
    cat_features = ['gender','embarked']
    train[cat_features] = train[cat_features].astype('category')
    test[cat_features] = test[cat_features].astype('category')


@reset_seeds
def preprocess_dataset():
    # 데이터 로드
    df_raw = __load_data()
    # 데이터 분리
    train, test = train_test_split(df_raw, test_size=0.2, stratify=df_raw['survived'])
    # 데이터 전처리
    __preprocess_data(train, test)
    # features, target 분리
    y = df_raw['survived']
    X = df_raw.drop(['survived'], axis=1)
    return train.drop(['survived'], axis=1), test.drop(['survived'], axis=1), train['survived'], test['survived']

Overwriting service/data.py


## model 생성

In [ ]:
%%writefile service/model.py

from lightgbm import LGBMClassifier, plot_importance
from utils import reset_seeds

# 모델 생성 후 리턴
@reset_seeds
def get_model(hp:dict=None, model_nm:str=None):
    if not hp:
        hp = {"verbose":-1} # warning 로그 제거

    if not model_nm:
        return LGBMClassifier(**hp)
    elif model_nm == "LGBMClassifier":
        return LGBMClassifier(**hp)

Overwriting service/model.py


## run 생성

In [ ]:
%%writefile service/run.py

import numpy as np
import pandas as pd

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import accuracy_score

from dataset import preprocess_dataset
from model import get_model
from utils import reset_seeds

def get_cross_validation(shuffle:bool=True, is_kfold:bool=True, n_splits:int=5):
    if is_kfold:
      return KFold(n_splits=n_splits, shuffle=shuffle)
    else:
      return StratifiedKFold(n_splits=n_splits, shuffle=shuffle)

def run_cross_validation(my_model, x_train, y_train, cv, is_kfold:bool=True):
    n_iter = 0
    accuracy_lst = []
    if is_kfold:
        cross_validation = cv.split(x_train)
    else:
        cross_validation = cv.split(x_train, y_train)

    for train_index, valid_index in cross_validation:
      n_iter += 1
      # 학습용, 검증용 데이터 구성
      train_x, valid_x = x_train.iloc[train_index], x_train.iloc[valid_index]
      train_y, valid_y = y_train.iloc[train_index], y_train.iloc[valid_index]
      # 학습
      my_model.fit(train_x, train_y)
      # 예측
      pred = my_model.predict(valid_x)
      # 평가
      accuracy = np.round(accuracy_score(valid_y, pred), 4)
      accuracy_lst.append(accuracy)
      print(f'{n_iter} 번째 K-fold 정확도: {accuracy}, 학습데이터 크기: {train_x.shape}, 검증데이터 크기: {valid_x.shape}')

    return np.mean(accuracy_lst)

@reset_seeds
def main():
    # 데이터 로드 및 분류
    X_train, X_test, y_train, y_test = preprocess_dataset()
    # 모델 생성
    my_model = get_model()
    # 교차 검증
    is_Regression = False
    my_cv = get_cross_validation(is_kfold=is_Regression)
    # 모델 학습
    accuracy = run_cross_validation(my_model, X_train, y_train, my_cv, is_kfold=is_Regression)

    # 테스트 데이터 예측
    return my_model.score(X_test, y_test)

if __name__=="__main__":
  result = main()
  print(f"테스트 스코어는 {result}")

Overwriting service/process.py


In [ ]:
!python3 service/run.py

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
before: (732, 12) / (184, 12)
1 번째 K-fold 정확도: 0.8435, 학습데이터 크기: (585, 8), 검증데이터 크기: (147, 8)
2 번째 K-fold 정확도: 0.8435, 학습데이터 크기: (585, 8), 검증데이터 크기: (147, 8)
3 번째 K-fold 정확도: 0.8493, 학습데이터 크기: (586, 8), 검증데이터 크기: (146, 8)
4 번째 K-fold 정확도: 0.8562, 학습데이터 크기: (586, 8), 검증데이터 크기: (146, 8)
5 번째 K-fold 정확도: 0.8425, 학습데이터 크기: (586, 8), 검증데이터 크기: (146, 8)
테스트 스코어는 0.875
